In [72]:
# config.py

class Config:
    PROD_DB = "postgresql://postgres:a1k8u2andre@" \
        "blastcards.cu9rorjtbkbf.sa-east-1.rds.amazonaws.com:5432/postgres"
    
    LOCAL_DB = "postgresql://blastcards:a1k8u2andre@" \
        "localhost:5433/blastcards"
    
    DATABASE = LOCAL_DB

config = Config()

In [73]:
# create_tables.py

import psycopg2
# from config import config

with psycopg2.connect(config.DATABASE) as conn:
    cur = conn.cursor()
    cur.execute("""
        create table if not exists tweets (
            id bigint primary key,
            username varchar not null,
            text varchar,
            detected timestamp
        )
    """)
    cur.execute("""
        create index idx_tweets_id on tweets(id);
    """)
    conn.commit()

DuplicateTable: relation "idx_tweets_id" already exists


In [74]:
# twitter.py
import psycopg2
from datetime import datetime
# from config import config
import requests

def make_auth_header():
    token = "AAAAAAAAAAAAAAAAAAAAAI3njgEAAAAA6XQNEZUNtTM4VbnNjZjTnjPuLts%3DN566JF8Lp5RwiSnbiG2dCWtG6Iv7Thr85s6C1Q7XIvGkf6wYyC" 
    return {"Authorization": f"Bearer {token}"}

def get_tweets_from_user(user: str):
    auth_header = make_auth_header()
    response = requests.get(f"https://api.twitter.com/2/tweets/search/recent?query=from:{user}", headers=auth_header)
    return response.json()

def save_tweet_on_database(user: str, tweet: dict) -> bool:
    """
    Return False if tweet was not saved (was saved before)
    Return True otherwise
    """
    if tweet_was_saved_before(tweet["id"]):
        return False
    with psycopg2.connect(config.DATABASE) as conn:
        cur = conn.cursor()
        cur.execute("""
             insert into tweets (id, username, text, detected) values
             (%s, %s, %s, %s)
        """, (tweet["id"], user, tweet["text"], datetime.now()))
        conn.commit()
    
    return True

def tweet_was_saved_before(tweet_id: int) -> bool:
    """
    Return True if tweet was saved before
    Return False otherwise
    """
    with psycopg2.connect(config.DATABASE) as conn:
        cur = conn.cursor()
        cur.execute("select id from tweets where id = %s", (tweet_id,))
        result = cur.fetchall()
        return False if len(result) == 0 else True


In [77]:
# users.py

import psycopg2
# from config import config

USERS = (
    "pythonjazz",
    "ike_sigma",
)

def get_last_user() -> str:
    with psycopg2.connect(config.DATABASE) as conn:
        cur = conn.cursor()
        cur.execute("select username, detected from tweets order by detected desc limit 1")
        result = cur.fetchone()
        return result[0]
    
def get_next_user() -> str:
    last_user = get_last_user()
    last_user_idx = USERS.index(last_user)
    if last_user_idx + 1 >= len(USERS):
        return USERS[0]
    
    return USERS[last_user_idx + 1]

In [80]:
# main.py

# import psycopg2
# from twitter import get_tweets_from_user
# from twitter import save_tweet_on_database
# from twitter import check_if_tweet_was_save_before
# from users import get_next_user

def main():
    user = get_next_user()
    print(user)
    tweets = get_tweets_from_user(user)
    print(tweets)
    for tweet in tweets["data"]:
        save_tweet_on_database("pythonjazz", tweet)

main()

ike_sigma
{'data': [{'edit_history_tweet_ids': ['1596626006315761665'], 'id': '1596626006315761665', 'text': '@zanfranceschi A minha mãe pegava era muito no inverno, fritava e Lepo !'}, {'edit_history_tweet_ids': ['1596621434847219715'], 'id': '1596621434847219715', 'text': 'RT @sseramemes: como identificar um brasileiro no stackoverflow https://t.co/dnovMWhAGX'}, {'edit_history_tweet_ids': ['1596621073533083648'], 'id': '1596621073533083648', 'text': 'Até o morcego treme o pintinho pra não ficar tudo respingado https://t.co/uqmoNUbfHe'}, {'edit_history_tweet_ids': ['1596620864522420224'], 'id': '1596620864522420224', 'text': 'RT @joaopauloz_: O Lionel Messi é cabuloso, temos que respeitar, meus amigos!'}, {'edit_history_tweet_ids': ['1596618992164212737'], 'id': '1596618992164212737', 'text': 'Falou tudo ! 🔝 https://t.co/du1DOrwNo6'}, {'edit_history_tweet_ids': ['1596575779336650752'], 'id': '1596575779336650752', 'text': 'Eu pareço normal, mas já tentei ver a olho nú a  a nebula da b